# 요약

- 개요 
    - 주식의 가격에는 다양한 요소들이 영향을 미치지만 그 중 뉴스에 민감하다고 판단하여 뉴스 기사의 제목을 분석하여 긍정/부정 평가를 한다. 


1. 데이터 수집
    - 최근 1일, 정확도 순서로 검색어 입력 시 그에 대한 뉴스 제목 정보를 크롤링한다.
    
2. 전처리
    - bert 모델로 제목에 대해서 형태소 분석을 하고 tokenize를 진행한다.
    - 제목을 수치화한 'sentiment' 값을 얻어내고 데이터는 아래와 같다.
    - train_data
       - feature : 미리 크롤링한 기사 제목에 대한 bert 모델을 적용한 수치값 (sentiment) (1과 0 사이의 유리수)
       - target : 부정/긍정. 0,1 (0은 부정 ,1은 긍정) (label)
           - train_data의 target은 직접 기사 제목을 읽고 라벨링을 수행함 
           - 일단 180개 정도 수행. 정확도를 올리기 위해 추가적인 라벨링 수행할 예정
    - test_data
       - feature : 1일간 크롤링한 기사 제목에 대한 bert 모델을 적용한 수치값 (sentiment)
       - target : 0,1 (label)
           - test_data의 target은 feature를 기반해서 0.5 이상이면 긍정, 0.5 미만이면 부정으로 설정   
        
3. 모델링
- bert 모델
    - 문장의 형태소 분석과, sentiment 값을 추출하기 위해 사용
- DecisionTree, RandomForestClassifier, LogisticRegression 모델 적용
    
4. 성능 평가
- 성능 평가를 위해 DecisionTree, RandomForestClassifier, LogisticRegression 회귀를 적용하여 




# 네이버 뉴스 크롤링
- 검색어 입력 받고 정확도순, 최근 1일 데이터 수집
- 제목 수준에서 긍/부정 판단

In [1]:
# 크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm

# 페이지 입력 (1 페이지당 기사 10개 이하)
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num + 1
    else:
        return num + 9 * (num - 1)


# search : 검색어, pd=4 : 최근 1일, start_page : 몇 페이지
def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        # 정확도순(디폴트)으로 1일간의 뉴스(pd=4) 
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search +"&pd=4"+"&start=" + str(
            start_page)
        return url
    else:
        # url 부분에서 정확도순서로 1일 데이터를 분류 가능
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search +"&pd=4"+"&start=" + str(page)
            urls.append(url)
        return urls

    # html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)

# 기사 내용 크롤링 함수
def news_attrs_crawler(articles, attrs):
    attrs_content = []
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content


# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

# html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    # html 불러오기
    original_html = requests.get(i, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select(
        "div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver, 'href')
    return url


#####뉴스크롤링 시작#####

# 테스트 횟수
num = int(input('테스트 횟수 입력 : '))
# 검색어 입력
search = input("검색 키워드 입력 : ")
# 검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):"))  # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ", page, "페이지")
# 검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):"))  # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ", page2, "페이지")

# naver url 생성
url = makeUrl(search, page, page2)

# 뉴스 크롤러 실행
news_titles = []
news_url = []

# 일단 제목 수준으로 진행 (기사 내용은 추후에 적용 여부 판단)
# news_contents = []

news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


# 제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist


# 제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

# 1차원 리스트로 만들기(내용 제외)
makeList(news_url_1, news_url)

# NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링
for i in tqdm(final_urls):
    # 각 기사 html get하기
    news = requests.get(i, headers=headers)
    news_html = BeautifulSoup(news.text, "html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
# ------------------------------------------------------------------------------------------------------------------
    # 뉴스 본문 가져오기 (일단 구현은 해놓음 but 일단 기사 제목 수준에서 진행)
#     content = news_html.select("div#dic_area")
#     if content == []:
#         content = news_html.select("#articeBody")

    # 기사 텍스트만 가져오기
    # list합치기
    #content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
#     content = re.sub(pattern=pattern1, repl='', string=content)
#     pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
#     content = content.replace(pattern2, '')
    
    news_titles.append(title)
#    news_contents.append(content)

    try:
        html_date = news_html.select_one(
            "div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1, repl='', string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ", (page2 + 1 - page) * 10, '개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
#print("\n[뉴스 내용]")
#print(news_contents)

print('news_title: ', len(news_titles))
print('news_url: ', len(final_urls))
#print('news_contents: ', len(news_contents))
print('news_dates: ', len(news_dates))

###데이터 프레임으로 만들기###
import pandas as pd

# 데이터 프레임 만들기
news_df = pd.DataFrame({'date': news_dates, 'title': news_titles})
# news_df = pd.DataFrame({'date': news_dates, 'title': news_titles, 'link': final_urls, 'content': news_contents})

# 중복 행 지우기
news_df = news_df.drop_duplicates(keep='first', ignore_index=True)
print("중복 제거 후 행 개수: ", len(news_df))

# 데이터 프레임 저장
now = datetime.datetime.now()
news_df.to_csv(f'csv/{search} 뉴스 제목{num}.csv', encoding='utf-8-sig', index=False)

print(type(news_df['title']))
# print(type(news_df['content']))


몇 번째 테스트? 1
검색할 키워드를 입력해주세요:삼성전자

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):10

크롤링할 종료 페이지:  10 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&pd=4&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&pd=4&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&pd=4&start=21', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&pd=4&start=31', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&pd=4&start=41', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&pd=4&start=51', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&pd=4&start=61', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&pd=4&start=71', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&pd=4&start=81', 'https://search.naver.com/search.naver?where=news&s

100%|██████████████████████████████████████████████████████████████████████████████████| 41/41 [00:07<00:00,  5.76it/s]


검색된 기사 갯수: 총  100 개

[뉴스 제목]
["삼성전자, 갤럭시·노트북·TV서 '자가 수리' 국내 도입", '삼성전자 갤럭시, 이제 내손으로 고친다', '[특징주] 삼성전자·SK하이닉스, 52주 신고가 기록', '"청년에 일경험"…SK하이닉스·삼성전자 등 2100명 모집', '작년 주주 배당액 1위는 삼성전자…총 배당금은 6.7%↓', '“고장난 갤럭시 직접 고친다”…삼성전자, ‘자가 수리’ 국내 도입', '"이번엔 중동"…삼성전자, 두바이 리조트서 2023년형 TV 공개', '삼성전자, 두바이서 2023년형 네오QLED 신제품 선봬', '[특징주] 날개 단 삼성전자‧SK하이닉스…또 52주 최고가 경신', '"이제 집에서 고치세요"...삼성전자, \'자가 수리\' 국내 도입', '"머스크 불만 터트릴 지경인데"…삼성전자·하이닉스 웃는 이유', '삼성전자 신고가 행진…"8만전자 간다"', '"삼성전자, 책임경영 필요성↑…이재용 등기이사 복귀 가능성"', "삼성전자, 외국인 폭풍매수에 '7만전자 굳히기'", '삼성전자, 7.1만원 진입…반도체株 훈풍 지속[특징주]', '"삼성전자가 또?"...신제품에 적용한 \'첨단 기술\'은?', '로이터 “삼성전자·LG전자, 베트남 정부에 최저한세 보상 요구”', '삼성전자 신고가 행진…코스피, 연중 최고치', '신고가 행진 삼성전자·SK하이닉스…"반도체 질주는 계속된다"', "'반갑다, 삼성전자'…반도체주 랠리에 1년만에 7만2000원선 회복", "삼성전자 '보행 보조 로봇' 특허 또 추가…'봇핏' 출시 임박?", "[증시신상품] 한화투자증권, '한화 델타랩 삼성전자우' 출시", '삼성전자·SK하이닉스 연고점…반도체주 랠리 이어질까', "삼성전자 vs TSMC…'AI 붐' 반도체 수혜주는 어디?", "코스피, 1년 만에 2580선…'파죽지세' 삼성전자 7.2만대", '[STOCK] "땡큐 엔비디아" 삼성전자·SK하이닉스, 나란히 신고가', "삼성전자 '8만전자'가 코앞?...52주 신고가 경신", '삼성전자·SK하이